# KaraFan v3.0
Colab version of MDX23 algorithm from [MVSEP.COM](https://www.mvsep.com) with some tweaks:
<font size=2>
Credits:
* [ZFTurbo/MVSep](https://github.com/ZFTurbo/MVSEP-MDX23-music-separation-model)
* Models by [Demucs](https://github.com/facebookresearch/demucs), [Anjok](https://github.com/Anjok07/ultimatevocalremovergui) & [Kimberley Jensen](https://github.com/KimberleyJensen)
* Adaptation & tweaks by [jarredou](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/)
</font>

In [ ]:
#@markdown #Installation
#@markdown *Run this cell to install and run KaraFan*

import os
import glob
import platform
import tempfile
from pathlib import Path

# Are we on Google Colab ?
Gdrive = ""; Temp = ""
try:
	from google.colab import drive
	drive.mount('/content/Gdrive')
	Gdrive = "/content/Gdrive/MyDrive"
	Temp = "/content"
except:
	system = platform.system()
	Gdrive = "C:\\Mon Drive" if system == "Windows" else "/mnt/Gdrive"  # "Linux", "Darwin" or "unknow"
	# If the system is not recognized (e.g., BSD), use /tmp as the default value.
	Temp = tempfile.gettempdir() if system in ["Windows", "Linux", "Darwin"] else "/tmp"

# Create directories
print("Temporary directory :", Temp)
Temp = os.path.join(Temp, "music_temp")
Project = os.path.join(Gdrive, "Github")
if not os.path.exists(Temp): os.mkdir(Temp)
if not os.path.exists(Project): os.mkdir(Project)

Project = os.path.join(Project, "KaraFan")
if not os.path.exists(Project):

	print("Installing... This will take 1 minute...")

	!git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2.git &> /dev/null

	os.chdir(Project)
	%pip install -r requirements.txt &> /dev/null

	print('Installation done !')

os.chdir(Project)

input         = '/content/drive/MyDrive/Music' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/Music_Results' #@param {type:"string"}
#@markdown ---
#@markdown <font size=2>*Bigshifts=1 to disable that feature*</font>
BigShifts_MDX = 11 #@param {type:"slider", min:1, max:41, step:1}
overlap_MDX = 0 #@param {type:"slider", min:0, max:0.95, step:0.05}
overlap_MDXv3 = 8 #@param {type:"slider", min:2, max:40, step:2}
#@markdown ---
weight_MDXv3 = 6 #@param {type:"slider", min:0, max:10, step:1}
weight_VOCFT = 5 #@param {type:"slider", min:0, max:10, step:1}
weight_HQ3 = 2 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
overlap_demucs = 0.6 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
output_format = 'PCM_16' #@param ["PCM_16", "FLOAT"]
vocals_instru_only = True #@param {type:"boolean"}
if vocals_instru_only:
	vocals_only = '--vocals_only true'
else:
	vocals_only = ''

#@markdown <font size=2 color=red>Use lower chunk_size value if you have memory errors !</font>
chunk_size = 500000 #@param {type:"slider", min:100000, max:1000000, step:100000}

if Path(input).is_file():
	file_path = input
	filename =  Path(input).stem
	Path(output_folder,filename).mkdir(parents=True, exist_ok=True)
	!python inference.py \
		--large_gpu \
		--weight_MDXv3 {weight_MDXv3} \
		--chunk_size {chunk_size} \
		--input_audio "{file_path}" \
		--overlap_demucs {overlap_demucs} \
		--overlap_MDX {overlap_MDX} \
		--overlap_MDXv3 {overlap_MDXv3} \
		--output_format {output_format} \
		--bigshifts {BigShifts_MDX} \
		--output_folder "{output_folder}/{filename}" \
		{vocals_only}
else:  
	for file_path in sorted(glob.glob(input+"/*"))[:]:
		filename =  Path(file_path).stem
		Path(output_folder,filename).mkdir(parents=True, exist_ok=True)
		!python inference.py \
			--large_gpu \
			--weight_MDXv3 {weight_MDXv3} \
			--chunk_size {chunk_size} \
			--input_audio "{file_path}" \
			--overlap_demucs {overlap_demucs} \
			--overlap_MDX {overlap_MDX} \
			--overlap_MDXv3 {overlap_MDXv3} \
			--output_format {output_format} \
			--bigshifts {BigShifts_MDX} \
			--output_folder "{output_folder}/{filename}" \
			{vocals_only}